In [51]:
# Importing the libraries
import os
import numpy as np
import pandas as pd

# Let's read the data
import numpy as np
sample = np.load('../data/x_train/0.npz')
data = sample['data']
print(data.shape)

# Let's see how many samples we have
print(f"X_train size : {len(os.listdir('../data/x_train'))}")
print(f"X_test size : {len(os.listdir('../data/x_test'))}")

(4, 128, 128)
X_train size : 100000
X_test size : 18149


In [52]:
# Here is the first benchmark
def benchmark(x_test_dir):
    n_t_out, out_size = 8, 2
    in_size = 128
    crop = (in_size - out_size) // 2
    benchmark_prediction = []
    benchmark_ids = []
    for file in os.listdir(x_test_dir):
        x_test = np.load(f'{x_test_dir}/{file}')
        y_bench = np.concatenate([
            x_test['data'][-1:, crop:-crop, crop:-crop] 
            for _ in range(n_t_out)
        ])
        benchmark_prediction.append(y_bench.mean(axis=(1, 2)))
        benchmark_ids.append(x_test['target_ids'])
    return pd.DataFrame({
        'ID': np.concatenate(benchmark_ids), 
        'TARGET': np.concatenate(benchmark_prediction)
    })

In [53]:
# Here is the second benchmark
get_method = None
extrapolation = None

def benchmark_pysteps(x_test_dir):
    n_t_out, out_size = 8, 2
    in_size = 128
    crop = (in_size - out_size) // 2
    benchmark_prediction = []
    benchmark_ids = []
    pysteps_flow_method = get_method('LK')
    for file in os.listdir(x_test_dir):

        x_test = np.load(f'{x_test_dir}/{file}')
        motion_field = pysteps_flow_method(x_test['data'])

        predictions = extrapolation.forecast(
            x_test['data'][-1, ...], 
            motion_field, 
            n_t_out
        )

        predictions[np.isnan(predictions)] = 0.

        benchmark_prediction.append(
            predictions[:, crop:-crop, crop:-crop].mean(axis=(1, 2))
        )

        benchmark_ids.append(x_test['target_ids'])
        
    return pd.DataFrame({
        'ID': np.concatenate(benchmark_ids), 
        'TARGET': np.concatenate(benchmark_prediction)
    })

It's seem we have a have enough data to train a neural network. There is spatial and space dependancies. We can try a convolutionnal model to extract features and then a recurrent model to predict the next frame. Or we could directly use a 3D convolutionnal layer to extract spatio-temporal features. I will try both.

In [54]:
# Let's first define commun class/function needed
import torch
import torch.nn as nn

# First i need a custom dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, x_dir, y_dir, transform=None):
        self.x_dir = x_dir
        self.y = pd.read_csv(f'../data/{y_dir}.csv')

    def __len__(self):
        return len(os.listdir(f'../data/{self.x_dir}'))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = np.load(f'../data/{self.x_dir}/{idx}.npz')['data']
        y = self.y[self.y['ID'] == idx]['TARGET'].values

        # Convert to PyTorch tensors
        x = torch.from_numpy(x).type(torch.float32)  # Assuming x is a numpy array
        y = torch.tensor(y, dtype=torch.float32)     # Adjust dtype as per your data

        return x, y


In [55]:
# Then i need a training loop
def train_loop(model, loss_fn, optimizer, train_loader, device):
    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [56]:
# Let's validate the model with evaluation metrics : Mean Squared Logarithmic Error
def test_loop(model, loss_fn, test_loader, device):
    size = len(test_loader.dataset)
    test_loss = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= size
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")


In [57]:
# Let's define loss, optimizer, device and training/testing data
# parameters
batch_size = 64
epochs = 5
learning_rate = 1e-3

# Loss function
loss_fn = nn.MSELoss()

# Optimizer
def opt(model, learning_rate, N):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    print(N)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=N)
    return optimizer, scheduler

# Device
device = torch.device('mps')

# Training data
split = 0.8
dataset = Dataset('x_train', 'y_train')

# Split data
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * split), len(dataset) - int(len(dataset) * split)])

# Data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [58]:
# Let's make a models
# I will begin with the CNN + LSTM model. With x[data] i have 4 time steps with features of 128x128.
# I need to use the same CNN to compute features from thoses 128x128 points.
# Then i will use a LSTM to learn the time dependency.
# I will then use a linear layer to predict the scalar value.

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        f1, f2, f3, f4 = 16, 16, 8, 4
        self.conv1 = nn.Conv2d(1, f1, 3)
        self.batchnorm1 = nn.BatchNorm2d(f1)
        self.conv2 = nn.Conv2d(f1, f2, 3)
        self.batchnorm2 = nn.BatchNorm2d(f2)
        self.conv3 = nn.Conv2d(f2, f3, 3)
        self.batchnorm3 = nn.BatchNorm2d(f3)
        self.conv4 = nn.Conv2d(f3, f4, 3)
        self.batchnorm4 = nn.BatchNorm2d(f4)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(4, -1, 1, 128, 128)
        x = [self.pool(self.relu(self.conv1(x[i]))) for i in range(4)]
        x = [self.batchnorm1(x[i]) for i in range(4)]
        x = [self.pool(self.relu(self.conv2(x[i]))) for i in range(4)]
        x = [self.batchnorm2(x[i]) for i in range(4)]
        x = [self.pool(self.relu(self.conv3(x[i]))) for i in range(4)]
        x = [self.batchnorm3(x[i]) for i in range(4)]
        x = [self.pool(self.relu(self.conv4(x[i]))) for i in range(4)]
        x = [self.batchnorm4(x[i]) for i in range(4)]
        x = torch.stack(x)
        x = x.view(-1, 4, 144)
        return x
    
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        self.gru = nn.GRU(144, 64, 1, batch_first=True)
        self.linear1 = nn.Linear(64, 32)
        self.linear2 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.cnn = CNN()

    def forward(self, x):
        x = self.cnn(x)
        x, _ = self.gru(x)
        x = x[:, -1, :]
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        return x

In [59]:
# Create metric for test purpose
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

In [60]:
# Let's train the model
batch_size = 64
epochs = 100
learning_rate = 1e-3
min_learning_rate = 1e-5
loss_fn = RMSLELoss()
model = GRU()

optimizer, scheduler = opt(model, learning_rate, len(train_loader))

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, loss_fn, optimizer, train_loader, device)
    test_loop(model, loss_fn, test_loader, device)
    scheduler.step()


1250


TypeError: '<=' not supported between instances of 'NoneType' and 'int'